In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 7, 30)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-07-30
yesterday: 2025-07-29


In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [9]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [12]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [69]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,WHAIR,156,2025-05-16,"10,000",4.70,1099,1320,2025-07-30,5.50,113.85,7.97,"54,878.18",75,"7,774.08",16.50%,80.32%,-99,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [71]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'WHAIR'

In [73]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1320



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,WHAIR,1320,156,2025-05-16,"10,000",4.70,97.29,6.81,Sold,"47,104.10",HD,None


In [75]:
transaction, name, qty, sell_price, buy_price

('S', 'WHAIR', -10000, 5.5, 4.7)

### In case of buy, retreive buy id from latest buy record (id)

In [18]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,WHART,1328,172,2025-07-08,"5,000",8.80,91.08,6.38,Active,"44,097.46",HD,None


In [20]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'WHART', 5000, 8.8)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [77]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'WHAIR'



,name,date,shares,price,active,period,grade
0,WHAIR,2021-10-11,"60,000",8.05,1,2,B1


In [79]:
print(transaction, name, qty, buy_price)

S WHAIR -10000 4.7


In [81]:
new_qty = buy['shares'] + qty
print(new_qty.iloc[0])

50000


In [83]:
new_price = 8.70
new_price

8.7

In [85]:
def update_buy(const, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [87]:
# Example usage
update_buy(const, name, new_qty, new_price)

'Records updated = 1'

In [89]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'WHAIR'



,name,date,shares,price,active,period,grade
0,WHAIR,2021-10-11,"50,000",8.70,1,2,B1


In [91]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [95]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'WHAIR'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHAIR,0.0582,0.2229,0.1372,0.1325,0.5508,"60,000",2025-05-26,2025-06-25,,0


In [97]:
new_qty

0    50000
Name: shares, dtype: int64

In [99]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [101]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [103]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'WHAIR'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHAIR,0.0582,0.2229,0.1372,0.1325,0.5508,"50,000",2025-05-26,2025-06-25,,0


In [105]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [109]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'WHAIR'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,127,WHAIR,0.00,0.00,B,4.30,8.05,0.00,0.00,8.05,"10,000",-4,4,"60,000",6.000000,0.000000,RD50pct,SET


In [111]:
# Extract scalar values from Pandas Series (if applicable)
new_qty_value = new_qty.iloc[0] if hasattr(new_qty, 'iloc') else new_qty
#cost_value = cost.iloc[0] if hasattr(cost, 'iloc') else cost
# Print the values
print(name, transaction, new_qty_value, new_price)

WHAIR S 50000 8.7


In [56]:
#print("Raw cost value before extraction:", cost)

In [113]:
def update_stock(conlite, name, new_qty, new_price):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty,
        cost = :new_price
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value, # Use the scalar value
        'new_price': new_price,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [115]:
# Example usage
update_stock(conlite, name, new_qty, new_price)

'Records updated = 1'

In [117]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'WHAIR'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,127,WHAIR,0.00,0.00,B,4.30,8.05,0.00,0.00,8.70,"10,000",-4,4,"50,000",6.000000,0.000000,RD50pct,SET


In [119]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM stocks
    WHERE name = :name AND available_qty = 0;
""")

# Execute the query with the correct parameter dictionary
result = conlite.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


In [121]:
conlite.commit()

### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [67]:
sql = """
SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = 'KCE'



,name,date,shares,price,active,dividend,period,grade,cost_amt
0,KCE,2021-10-07,3000,87.0,1,1.2,4,C2,261000.0


In [69]:
print(transaction, name, qty, buy_price)

S KCE -1000 87.0


In [26]:
new_unit_cost = buy['cost_amt'].iloc[0] + (qty * buy_price)
new_qty = buy['shares'].iloc[0] + qty
print(new_unit_cost)
print(new_qty)
new_unit_cost = new_unit_cost / new_qty
print(new_unit_cost)

290000.0
25000
11.6


### In case there is the same stock name in port

In [28]:
def update_buy(const, name, new_qty, new_unit_cost):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty
    new_unit_cost_value = new_unit_cost
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty, 
            price = :new_unit_cost
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [30]:
# Example usage
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [32]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'WHART'



,name,date,shares,price,active,dividend,period,grade
0,WHART,2022-01-05,"25,000",11.60,1,0.7133,2,C1


### In case of totally new stock

In [11]:
rcd = ['TVO', '2024-11-06', 4000, 24.90, 0, 0, 1, 0, 4,'A2']
rcd

['TVO', '2024-11-06', 4000, 24.9, 0, 0, 1, 0, 4, 'A2']

In [12]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [13]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [15]:
name = 'TVO'

In [16]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'TVO'



,name,date,shares,price,active,dividend,period,grade
0,TVO,2024-11-06,"4,000",24.90,1,0.0000,4,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [34]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'WHART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1358,0.1930,0.1930,0.1915,0.7133,"20,000",2025-05-19,2025-06-06,,0


In [36]:
new_qty = dividend['shares'].iloc[0] + qty
print(new_qty)

25000


In [38]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [40]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [42]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'WHART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1358,0.1930,0.1930,0.1915,0.7133,"25,000",2025-05-19,2025-06-06,,0


### In case of totally new stock

In [15]:
rcd = ['TVO', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]
rcd

['TOA', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]

In [15]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [16]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TOA' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TOA,0.2500,0.0000,0.3300,0.0000,0.5800,"1,000",2023-08-30,2023-09-14,,1


### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [44]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,149,WHART,0.0,0.0,B,8.8,12.1,0.0,0.0,12.3,5000,-4,4,20000,8.6,11.74,RD30Pct,SET


In [46]:
#
new_buy_target = 8.00
new_sell_target = 9.7
new_buy_qty = 5_000
print(transaction, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

B WHART 5000 8.8 8.0 9.7 5000


In [48]:
print(new_qty)

25000


In [50]:
print(new_unit_cost)

11.6


In [52]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty)
    new_unit_cost_value = float(new_unit_cost)   
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [54]:
# Example usage
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [56]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,149,WHART,0.0,0.0,B,8.0,9.7,0.0,0.0,11.6,5000,-4,4,25000,8.6,11.74,RD30Pct,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

### =================================================

In [123]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-07-29' ORDER BY period, buy.name


In [125]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,14.80,"44,400.00","17,760.00","-26,640.00",-60.00
1,1,A1,AWC,2023-06-15,"9,000",4.96,2.08,"44,640.00","18,720.00","-25,920.00",-58.06
2,1,A2,JMART,2023-01-20,"6,800",33.00,7.55,"224,400.00","51,340.00","-173,060.00",-77.12
3,1,C2,ORI,2022-04-07,"60,000",8.98,1.90,"538,500.00","114,000.00","-424,500.00",-78.83
4,1,C2,PTTGC,2021-03-17,"6,000",64.75,23.90,"388,500.00","143,400.00","-245,100.00",-63.09
5,1,C2,SCC,2021-09-20,600,405.00,205.00,"243,000.00","123,000.00","-120,000.00",-49.38
6,1,A3,SINGER,2023-01-19,"6,000",24.80,5.20,"148,800.00","31,200.00","-117,600.00",-79.03
7,2,A1,AIMIRT,2023-08-17,"12,500",10.90,9.55,"136,250.00","119,375.00","-16,875.00",-12.39
8,2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.50,"990,000.00","632,500.00","-357,500.00",-36.11
9,2,C1,DIF,2020-08-01,"40,000",13.10,8.15,"524,000.00","326,000.00","-198,000.00",-37.79


In [127]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [129]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [131]:
output.to_csv(output_file, header=True, index=False)
output.to_csv(icd_file, header=True, index=False)
output.to_csv(god_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)

In [133]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

output.to_excel(xsl_file, index=False)

In [135]:
conlite.commit()
conlite.close()